In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from skimage.filters import threshold_mean
import sys

sys.path.append('../')
from src.functions import get_parameters
from src.functions import get_key_file

In [2]:
parameter_local_path = '../conf/local/parameters.yml'

parameters = get_parameters(parameter_local_path)

base_folder = parameters["data_dir"]
out_folder = parameters["out_dir"]
key_file_path = parameters["key_file"]

plot_width = 7
plot_height = 7
channel_artery = 1

key_file = get_key_file(parameters)

print(key_file.columns)
display(key_file)

#filename_GFPxERG = base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Erg X GFP.tif"
#filename_ERG = base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Erg_threshold.ome.tif"
#filename_drawn =  base_folder + "/VEGFR3Cre mTmG/P5_P6/retina1/Drawn.ome.tiff"

Parameters: 
{'key_file': '/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/2024-04-05_keyfile.xls', 'data_dir': '/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/', 'out_dir': '/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/output/', 'pixel_no_per_retina': 10000, 'image_types': ['GFP'], 'load_conditions': ['CDH5CreERT2;iSureCre;YesFloxFlox', 'CDH5CreERT2 iSureCre'], 'plot_kde': False, 'plot_individual_hist': False, 'plot_cond_kde': True, 'radial_bins': [[0, 1200]], 'time_points': ['P3_P7', 'P3_P15'], 'number_ks_test': 1}
Index(['filename', 'ExperimentID', 'Cdh5CreERT2 mTmG',
       'Cdh5CreERT2;mTmG;SMAD4FloxFlox', 'Cdh5CreERT2;mTmG;Alk1FloxFlox',
       'Drawn', 'Unnamed: 6', 'GFP', 'Unnamed: 8', 'IB4',
       'THO Injection Point', 'Collection Point', 'Pixel size in um'],
      dtype='object')


,filename,ExperimentID,Cdh5CreERT2 mTmG,Cdh5CreERT2;mTmG;SMAD4FloxFlox,Cdh5CreERT2;mTmG;Alk1FloxFlox,Drawn,Unnamed: 6,GFP,Unnamed: 8,IB4,THO Injection Point,Collection Point,Pixel size in um
0,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,1,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
1,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,1,0,0,1,1,NaN,0,NaN,0,5,8,2.273761
2,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,2,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
3,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,2,0,0,1,1,NaN,0,NaN,0,5,8,2.273761
4,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,3,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P1...,53,0,0,1,1,NaN,0,NaN,0,5,15,2.273761
106,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,54,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
107,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,54,0,0,1,1,NaN,0,NaN,0,5,8,2.273761
108,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,55,0,0,1,0,NaN,1,NaN,0,5,8,2.273761


In [3]:
summary_data = pd.DataFrame()
counter = 0

experiment_IDs = key_file["ExperimentID"].unique() #[70:]
#experiment_IDs = [10]

experiment_IDs_exclude = []

for experiment_ID in experiment_IDs:
    print(experiment_ID)
    if experiment_ID in experiment_IDs_exclude:
        print("Excluded experiment ID: %s" % experiment_ID)
        continue
    experiment_key = key_file[key_file["ExperimentID"] == experiment_ID]
    
    
    
    display(experiment_key)

    filename_GFP = base_folder
    filename_drawn = base_folder
    
    count_img = 0
    for index, row in experiment_key.iterrows():
        
        if row["GFP"] == 1:
            filename_GFP = base_folder + row["filename"]
            count_img += 1
        if row["Drawn"] == 1:
            filename_drawn =  base_folder + row["filename"]
            count_img += 1
            
    if count_img != 2:
        print("Need exactly 2 images, but %s were given!" % count_img)
        continue
    
    print(filename_GFP)
    print(filename_drawn)
    
    img_GFP = np.array(io.imread(filename_GFP))
    #print(img_GFPxERG.shape)
    #print(threshold_mean(img_GFPxERG))
    thresh_img_GFP = img_GFP > threshold_mean(img_GFP)
    unique_GFP, counts_GFP = np.unique(thresh_img_GFP, return_counts = True)    
    if counts_GFP[1] > counts_GFP[0]:
        thresh_img_GFP = img_GFP < threshold_mean(img_GFP)
        unique_GFP, counts_GFP = np.unique(thresh_img_GFP, return_counts = True)
        #counts_GFPxERG = np.count_nonzero(thresh_img_GFPxERG)
    

    img_drawn = np.array(io.imread(filename_drawn))
    print("Shape of drawn image:")
    print(img_drawn.shape)
    img_artery = img_drawn[1,:,:]
    #print(threshold_mean(img_artery))
    thresh_img_artery = img_artery > threshold_mean(img_artery)
    GFP_artery = np.logical_and(thresh_img_GFP,thresh_img_artery)
    
    img_vein = img_drawn[2,:,:]
    #print(threshold_mean(img_artery))
    thresh_img_vein = img_vein > threshold_mean(img_vein)
    GFP_vein = np.logical_and(thresh_img_GFP,thresh_img_vein)
    
    #unique_ERG, counts_ERG = np.unique(thresh_img_ERG, return_counts = True)
    #unique_GFP_artery, counts_GFP_artery = np.unique(GFP_artery, return_counts = True)
    #unique_GFP_vein, counts_GFP_vein = np.unique(GFP_vein, return_counts = True)
    
    counts_GFP_artery = np.count_nonzero(GFP_artery)
    counts_GFP_vein = np.count_nonzero(GFP_vein)
    
    
    #print(counts_ERG)
    #print(counts_GFPxERG)
    #print(100.0*counts_GFPxERG[1]/counts_ERG[1])
    #print(100.0*counts_GFP_artery[1]/counts_ERG[1])
    #print(100.0*counts_GFP_artery[1]/counts_GFPxERG[1])
    
    
    summary_data.at[counter,"ExperimentID"] = experiment_ID
    #summary_data.at[counter,"condition"] = row["condition"]
    #summary_data.at[counter,"time_interval"] = row["time_interval"]
    #summary_data.at[counter,"ERG_px"] = counts_ERG[1]
    summary_data.at[counter,"collection_time"] = row["Collection Point"]
    summary_data.at[counter,"GFP_px"] = counts_GFP[1]
    summary_data.at[counter,"GFP_artery_px"] = counts_GFP_artery
    summary_data.at[counter,"GFP_vein_px"] = counts_GFP_vein
    
    summary_data.to_csv(out_folder + "summary_data.csv", index=False)
    
    fig, ax = plt.subplots(3,2, figsize=(2*plot_width, 3*plot_height))
    #ax[0,0].imshow(thresh_img_ERG)
    ax[0,0].set_title("IB4 channel (need to add)")
    ax[0,1].imshow(thresh_img_GFP)
    ax[0,1].set_title("GFP channel")
    ax[1,0].imshow(thresh_img_artery)
    ax[1,0].set_title("Artery mask")
    ax[1,1].imshow(GFP_artery)
    ax[1,1].set_title("GFP in artery")
    ax[2,0].imshow(thresh_img_vein)
    ax[2,0].set_title("Vein mask")
    ax[2,1].imshow(GFP_vein)
    ax[2,1].set_title("GFP in vein")
    plt.savefig(out_folder + "validation_images/masks_experiment_id_%s.png" % experiment_ID)
    plt.close()
    
    counter += 1    

1


,filename,ExperimentID,Cdh5CreERT2 mTmG,Cdh5CreERT2;mTmG;SMAD4FloxFlox,Cdh5CreERT2;mTmG;Alk1FloxFlox,Drawn,Unnamed: 6,GFP,Unnamed: 8,IB4,THO Injection Point,Collection Point,Pixel size in um
0,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,1,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
1,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,1,0,0,1,1,NaN,0,NaN,0,5,8,2.273761


/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8/20240215_p5_p8_alk1_cdh5_mtmg_1_Merged/output_retina_1/GFP_mask.ome.tif
/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8/20240215_p5_p8_alk1_cdh5_mtmg_1_Merged/output_retina_1/Drawn.ome.tif
Shape of drawn image:
(5, 2351, 2351)
2


,filename,ExperimentID,Cdh5CreERT2 mTmG,Cdh5CreERT2;mTmG;SMAD4FloxFlox,Cdh5CreERT2;mTmG;Alk1FloxFlox,Drawn,Unnamed: 6,GFP,Unnamed: 8,IB4,THO Injection Point,Collection Point,Pixel size in um
2,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,2,0,0,1,0,NaN,1,NaN,0,5,8,2.273761
3,migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8...,2,0,0,1,1,NaN,0,NaN,0,5,8,2.273761


/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8/20240215_p5_p8_alk1_cdh5_mtmg_2_Merged/output_retina_2/GFP_mask.ome.tif
/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/Alk1_mTmG/P8/20240215_p5_p8_alk1_cdh5_mtmg_2_Merged/output_retina_2/Drawn.ome.tif
Shape of drawn image:
(5, 2349, 2349)
3


...

/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_3_Merged/output_retina_3/GFP_mask.ome.tif
/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_3_Merged/output_retina_3/Drawn.ome.tif
Shape of drawn image:
(5, 2371, 2377)
19


,filename,ExperimentID,Cdh5CreERT2 mTmG,Cdh5CreERT2;mTmG;SMAD4FloxFlox,Cdh5CreERT2;mTmG;Alk1FloxFlox,Drawn,Unnamed: 6,GFP,Unnamed: 8,IB4,THO Injection Point,Collection Point,Pixel size in um
36,migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P...,19,0,1,0,0,NaN,1,NaN,0,5,15,2.273761
37,migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P...,19,0,1,0,1,NaN,0,NaN,0,5,15,2.273761


/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_4_Merged/output_retina_4/GFP_mask.ome.tif
/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_4_Merged/output_retina_4/Drawn.ome.tif
Shape of drawn image:
(5, 2372, 2378)
20


,filename,ExperimentID,Cdh5CreERT2 mTmG,Cdh5CreERT2;mTmG;SMAD4FloxFlox,Cdh5CreERT2;mTmG;Alk1FloxFlox,Drawn,Unnamed: 6,GFP,Unnamed: 8,IB4,THO Injection Point,Collection Point,Pixel size in um
38,migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P...,20,0,1,0,0,NaN,1,NaN,0,5,15,2.273761
39,migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P...,20,0,1,0,1,NaN,0,NaN,0,5,15,2.273761


/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_5_Merged/output_retina_5/GFP_mask.ome.tif
/media/fjodor/Elephant/Dropbox/MDC_Projects/Manuscript_HHT/migration_assay/in_vivo/mosaic_EC/SMAD4_mTmG/P15/20240311_p15_s4_cdh5_mtmg_5_Merged/output_retina_5/Drawn.ome.tif
Shape of drawn image:
(5, 2370, 2378)
